This module works for HOURLY data:

    ** 1. Merge the pumped signals with time stamps to get vertical format for voting
    ** 2. Append the output whethere the coin was pumped or not (binary 0)

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

## Import the pumped data

### Step 1. Create the timestamp for the first hours 

In [2]:
p = pd.read_pickle("../all_pumps.pkl")
# p = p[['channelLink','channelTitle','currency','exchange','signalTime','pumptime_edited','timestamp']]
p.head(2)

,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time
0,NAV,Binance,2019-12-19 17:00:15+00:00,VIB,1576774815,2019-12-19,17:00:15
1,NAV,Binance,2019-12-19 17:00:15+00:00,STEEM,1576774815,2019-12-19,17:00:15


Ther are .... unique points in time...

In [3]:
#Number of unique time points
len(pd.unique(p['timestamp']))

481

In [4]:
#Number of unique PUMPED coins points
len(pd.unique(p['currency']))

109

In [5]:
#Number of unique currency allowed
len(pd.unique(p['coin_name']))

123

In [6]:
1562346032 - (1562346032 % (86400))

1562284800

..Strange beheavior, must write each function one by one...

In [7]:
def getHourstamp(dfo: pd.DataFrame, num_hours: int=1, var:str='timestamp'):
    '''
    Append a column that contains the timestamp at 12:00 AM (0:00) of the
    original var variable.
    :param df: dataframe in question 
    :param num_date: number of dates to go back to
    :param var str: name of the variable that has original timestamp
    '''
    df = dfo.copy()
    df[var+'_int'] = df[var].apply(int)
    for d in range(num_hours+1):
        df[var+'_h'+str(d)] = df[var+'_int'].apply(lambda t : t - (t % (3600)) - d*(3600))
    df = df.drop(columns=var+'_int')
    return df

In [8]:
#We leave it for 1 days in advance
num_hours = 48
p = getHourstamp(p,num_hours)
p.head()

,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48
0,NAV,Binance,2019-12-19 17:00:15+00:00,VIB,1576774815,2019-12-19,17:00:15,1576774800,1576771200,1576767600,1576764000,1576760400,1576756800,1576753200,1576749600,1576746000,1576742400,1576738800,1576735200,1576731600,1576728000,1576724400,1576720800,1576717200,1576713600,1576710000,1576706400,1576702800,1576699200,1576695600,1576692000,1576688400,1576684800,1576681200,1576677600,1576674000,1576670400,1576666800,1576663200,1576659600,1576656000,1576652400,1576648800,1576645200,1576641600,1576638000,1576634400,1576630800,1576627200,1576623600,1576620000,1576616400,1576612800,1576609200,1576605600,1576602000
1,NAV,Binance,2019-12-19 17:00:15+00:00,STEEM,1576774815,2019-12-19,17:00:15,1576774800,1576771200,1576767600,1576764000,1576760400,1576756800,1576753200,1576749600,1576746000,1576742400,1576738800,1576735200,1576731600,1576728000,1576724400,1576720800,1576717200,1576713600,1576710000,1576706400,1576702800,1576699200,1576695600,1576692000,1576688400,1576684800,1576681200,1576677600,1576674000,1576670400,1576666800,1576663200,1576659600,1576656000,1576652400,1576648800,1576645200,1576641600,1576638000,1576634400,1576630800,1576627200,1576623600,1576620000,1576616400,1576612800,1576609200,1576605600,1576602000
2,NAV,Binance,2019-12-19 17:00:15+00:00,NEBL,1576774815,2019-12-19,17:00:15,1576774800,1576771200,1576767600,1576764000,1576760400,1576756800,1576753200,1576749600,1576746000,1576742400,1576738800,1576735200,1576731600,1576728000,1576724400,1576720800,1576717200,1576713600,1576710000,1576706400,1576702800,1576699200,1576695600,1576692000,1576688400,1576684800,1576681200,1576677600,1576674000,1576670400,1576666800,1576663200,1576659600,1576656000,1576652400,1576648800,1576645200,1576641600,1576638000,1576634400,1576630800,1576627200,1576623600,1576620000,1576616400,1576612800,1576609200,1576605600,1576602000
3,NAV,Binance,2019-12-19 17:00:15+00:00,WABI,1576774815,2019-12-19,17:00:15,1576774800,1576771200,1576767600,1576764000,1576760400,1576756800,1576753200,1576749600,1576746000,1576742400,1576738800,1576735200,1576731600,1576728000,1576724400,1576720800,1576717200,1576713600,1576710000,1576706400,1576702800,1576699200,1576695600,1576692000,1576688400,1576684800,1576681200,1576677600,1576674000,1576670400,1576666800,1576663200,1576659600,1576656000,1576652400,1576648800,1576645200,1576641600,1576638000,1576634400,1576630800,1576627200,1576623600,1576620000,1576616400,1576612800,1576609200,1576605600,1576602000
4,NAV,Binance,2019-12-19 17:00:15+00:00,FTM,1576774815,2019-12-19,17:00:15,1576774800,1576771200,1576767600,1576764000,1576760400,1576756800,1576753200,1576749600,1576746000,1576742400,1576738800,1576735200,1576731600,1576728000,1576724400,1576720800,1576717200,1576713600,1576710000,1576706400,1576702800,1576699200,1576695600,1576692000,1576688400,1576684800,1576681200,1576677600,1576674000,1576670400,1576666800,1576663200,1576659600,1576656000,1576652400,1576648800,1576645200,1576641600,1576638000,1576634400,1576630800,1576627200,1576623600,1576620000,1576616400,1576612800,1576609200,1576605600,1576602000


### Step 2: Loop throug the processed social data file and merge based on timestemp
... We do not need to join by the coinID. Just the timestamps are good enough. 
Since we want to retrieve the values for all availabel timestamps

## TEST A SINGLE SOCIAL DATA

In [9]:
test = pd.read_pickle("../../Social_Data/Hourly/Hourly_VIB_198710.pkl")
test.head(2)

,time,comments,posts,followers,points,overview_page_views,analysis_page_views,markets_page_views,charts_page_views,trades_page_views,forum_page_views,influence_page_views,total_page_views,fb_likes,fb_talking_about,twitter_followers,twitter_following,twitter_lists,twitter_favourites,twitter_statuses,reddit_subscribers,reddit_active_users,reddit_posts_per_hour,reddit_posts_per_day,reddit_comments_per_hour,reddit_comments_per_day,code_repo_stars,code_repo_forks,code_repo_subscribers,code_repo_open_pull_issues,code_repo_closed_pull_issues,code_repo_open_issues,code_repo_closed_issues,code_repo_contributors,date,date_utc
0,1520154000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,2018-03-04 01:00:00,2018-03-04 09:00:00+00:00
1,1520157600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,2018-03-04 02:00:00,2018-03-04 10:00:00+00:00


## HOURLY
Note: we want to work with the EDITED timesries, not the raw social data.

In [10]:
dir_prefix = "../../Social_Data/Hourly/Processed Data/Time Series/"
# dir_prefix = "../../Social_Data/Hourly/"
social_error = []
x_final = pd.DataFrame()
t_temp = pd.DataFrame()
for filename in os.listdir(dir_prefix):
    if 'pkl' in filename and 'error' not in filename:
        print(filename)
        _, coin_name, coin_id,_= filename.split('_')
        exchange = "Binance"
        #There is no exchang here
        try: 
            print("Processing file...",filename)
            # Must restrict only to the coins in the pumped list...
            l = p[(p.coin_name == coin_name) & (p.exchange == exchange)].copy()
            for d in range(num_hours+1):
                if 'timestamp_h'+str(d) not in l.columns: 
                    print('Info. on hour_{} not in the left table!'.format(d))
                    continue
                x = pd.read_pickle(dir_prefix+filename)
                x.columns = [x+'_h'+str(d) if x != 'time' else 'time' for x in x.columns]
                l = pd.merge(left=l
                             ,left_on=['timestamp_h'+str(d)]
                             ,right=x, right_on=["time"]
                             ,how='inner')
                l = l.drop(columns=['time'])
            #Save the file in the processed_directory
            l['coin_name'] = coin_name
            l['coin_id'] = coin_id.replace('.pkl','')
            #Append all the data on top of each other for processing; 
            x_final = x_final.append(l)
        except:
            e = sys.exc_info()[0]
            print( "<p>Error: %s</p>" % e )
            social_error.append(filename)
            continue

Hourly_ALEX_45607_Time.pkl
Processing file... Hourly_ALEX_45607_Time.pkl
Hourly_MET_916769_Time.pkl
Processing file... Hourly_MET_916769_Time.pkl
Hourly_VIBE_336538_Time.pkl
Processing file... Hourly_VIBE_336538_Time.pkl
Hourly_CHAN_175683_Time.pkl
Processing file... Hourly_CHAN_175683_Time.pkl
Hourly_SPX_683963_Time.pkl
Processing file... Hourly_SPX_683963_Time.pkl
Hourly_EQT_105263_Time.pkl
Processing file... Hourly_EQT_105263_Time.pkl
Hourly_DLT_220204_Time.pkl
Processing file... Hourly_DLT_220204_Time.pkl
Hourly_MCI_338415_Time.pkl
Processing file... Hourly_MCI_338415_Time.pkl
Hourly_DAS_194399_Time.pkl
Processing file... Hourly_DAS_194399_Time.pkl
Hourly_PTOY_129035_Time.pkl
Processing file... Hourly_PTOY_129035_Time.pkl
Hourly_DAI_932637_Time.pkl
Processing file... Hourly_DAI_932637_Time.pkl
Hourly_BERN_20817_Time.pkl
Processing file... Hourly_BERN_20817_Time.pkl
Hourly_DIM_180678_Time.pkl
Processing file... Hourly_DIM_180678_Time.pkl
Hourly_PIVX_42433_Time.pkl
Processing file...

Hourly_GUN_645936_Time.pkl
Processing file... Hourly_GUN_645936_Time.pkl
Hourly_GNO_66193_Time.pkl
Processing file... Hourly_GNO_66193_Time.pkl
Hourly_NANO_172091_Time.pkl
Processing file... Hourly_NANO_172091_Time.pkl
Hourly_ALGO_930992_Time.pkl
Processing file... Hourly_ALGO_930992_Time.pkl
Hourly_WTL_926317_Time.pkl
Processing file... Hourly_WTL_926317_Time.pkl
Hourly_OK_17857_Time.pkl
Processing file... Hourly_OK_17857_Time.pkl
Hourly_ALIS_331603_Time.pkl
Processing file... Hourly_ALIS_331603_Time.pkl
Hourly_KTK_5311_Time.pkl
Processing file... Hourly_KTK_5311_Time.pkl
Hourly_NETKO_53174_Time.pkl
Processing file... Hourly_NETKO_53174_Time.pkl
Hourly_CNO_42438_Time.pkl
Processing file... Hourly_CNO_42438_Time.pkl
Hourly_BSV_926591_Time.pkl
Processing file... Hourly_BSV_926591_Time.pkl
Hourly_BTT_928723_Time.pkl
Processing file... Hourly_BTT_928723_Time.pkl
Hourly_WAN_240142_Time.pkl
Processing file... Hourly_WAN_240142_Time.pkl
Hourly_PNC_33873_Time.pkl
Processing file... Hourly_PNC

Hourly_KMD_26132_Time.pkl
Processing file... Hourly_KMD_26132_Time.pkl
Hourly_SUB_221463_Time.pkl
Processing file... Hourly_SUB_221463_Time.pkl
Hourly_SRN_380641_Time.pkl
Processing file... Hourly_SRN_380641_Time.pkl
Hourly_HDG_237407_Time.pkl
Processing file... Hourly_HDG_237407_Time.pkl
Hourly_APX_74912_Time.pkl
Processing file... Hourly_APX_74912_Time.pkl
Hourly_XID_199026_Time.pkl
Processing file... Hourly_XID_199026_Time.pkl
Hourly_PLR_184298_Time.pkl
Processing file... Hourly_PLR_184298_Time.pkl
Hourly_GRN_931450_Time.pkl
Processing file... Hourly_GRN_931450_Time.pkl
Hourly_UGAS_929793_Time.pkl
Processing file... Hourly_UGAS_929793_Time.pkl
Hourly_FET_929174_Time.pkl
Processing file... Hourly_FET_929174_Time.pkl
Hourly_UKG_371005_Time.pkl
Processing file... Hourly_UKG_371005_Time.pkl
Hourly_RLC_61877_Time.pkl
Processing file... Hourly_RLC_61877_Time.pkl
Hourly_DAI_769546_Time.pkl
Processing file... Hourly_DAI_769546_Time.pkl
Hourly_LDC_732021_Time.pkl
Processing file... Hourly_LD

In [11]:
print(social_error)
print(l.shape)

[]
(0, 1870)


#We might have to check whether there is a non-zero data somewhere....

In [12]:
x_final = x_final.reset_index(drop=True)
x_final = x_final.sort_values('timestamp')
x_final = x_final.drop_duplicates()

In [13]:
x_final.filter(regex="twitter_fav")

,twitter_favourites_h0,twitter_favourites_h1,twitter_favourites_h2,twitter_favourites_h3,twitter_favourites_h4,twitter_favourites_h5,twitter_favourites_h6,twitter_favourites_h7,twitter_favourites_h8,twitter_favourites_h9,twitter_favourites_h10,twitter_favourites_h11,twitter_favourites_h12,twitter_favourites_h13,twitter_favourites_h14,twitter_favourites_h15,twitter_favourites_h16,twitter_favourites_h17,twitter_favourites_h18,twitter_favourites_h19,twitter_favourites_h20,twitter_favourites_h21,twitter_favourites_h22,twitter_favourites_h23,twitter_favourites_h24,twitter_favourites_h25,twitter_favourites_h26,twitter_favourites_h27,twitter_favourites_h28,twitter_favourites_h29,twitter_favourites_h30,twitter_favourites_h31,twitter_favourites_h32,twitter_favourites_h33,twitter_favourites_h34,twitter_favourites_h35,twitter_favourites_h36,twitter_favourites_h37,twitter_favourites_h38,twitter_favourites_h39,twitter_favourites_h40,twitter_favourites_h41,twitter_favourites_h42,twitter_favourites_h43,twitter_favourites_h44,twitter_favourites_h45,twitter_favourites_h46,twitter_favourites_h47,twitter_favourites_h48
32617,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41293,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53343,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29725,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21207,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882,1882
20725,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20243,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717,717
58803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


*** Now we have the time series data up to 1 month in the past!
*** So now we can start preparing the data and sort the categories together?

In [14]:
print(x_final.shape)
x_final[-5:]

(59286, 1870)


,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48,comments_h0,posts_h0,followers_h0,points_h0,overview_page_views_h0,analysis_page_views_h0,markets_page_views_h0,charts_page_views_h0,trades_page_views_h0,forum_page_views_h0,influence_page_views_h0,total_page_views_h0,fb_likes_h0,fb_talking_about_h0,twitter_followers_h0,twitter_following_h0,twitter_lists_h0,twitter_favourites_h0,twitter_statuses_h0,reddit_subscribers_h0,reddit_active_users_h0,reddit_posts_per_hour_h0,reddit_posts_per_day_h0,reddit_comments_per_hour_h0,reddit_comments_per_day_h0,code_repo_stars_h0,code_repo_forks_h0,code_repo_subscribers_h0,code_repo_open_pull_issues_h0,code_repo_closed_pull_issues_h0,code_repo_open_issues_h0,code_repo_closed_issues_h0,code_repo_contributors_h0,date_h0,date_utc_h0,coin_name_h0,coin_id_h0,comments_h1,posts_h1,followers_h1,points_h1,overview_page_views_h1,analysis_page_views_h1,markets_page_views_h1,charts_page_views_h1,trades_page_views_h1,forum_page_views_h1,influence_page_views_h1,total_page_views_h1,fb_likes_h1,fb_talking_about_h1,twitter_followers_h1,twitter_following_h1,twitter_lists_h1,twitter_favourites_h1,twitter_statuses_h1,reddit_subscribers_h1,reddit_active_users_h1,reddit_posts_per_hour_h1,reddit_posts_per_day_h1,reddit_comments_per_hour_h1,reddit_comments_per_day_h1,code_repo_stars_h1,code_repo_forks_h1,code_repo_subscribers_h1,code_repo_open_pull_issues_h1,code_repo_closed_pull_issues_h1,code_repo_open_issues_h1,code_repo_closed_issues_h1,code_repo_contributors_h1,date_h1,date_utc_h1,coin_name_h1,coin_id_h1,comments_h2,posts_h2,followers_h2,points_h2,overview_page_views_h2,analysis_page_views_h2,markets_page_views_h2,charts_page_views_h2,trades_page_views_h2,forum_page_views_h2,influence_page_views_h2,total_page_views_h2,fb_likes_h2,fb_talking_about_h2,twitter_followers_h2,twitter_following_h2,twitter_lists_h2,twitter_favourites_h2,twitter_statuses_h2,reddit_subscribers_h2,reddit_active_users_h2,reddit_posts_per_hour_h2,reddit_posts_per_day_h2,reddit_comments_per_hour_h2,reddit_comments_per_day_h2,code_repo_stars_h2,code_repo_forks_h2,code_repo_subscribers_h2,code_repo_open_pull_issues_h2,code_repo_closed_pull_issues_h2,code_repo_open_issues_h2,code_repo_closed_issues_h2,code_repo_contributors_h2,date_h2,date_utc_h2,coin_name_h2,coin_id_h2,comments_h3,posts_h3,followers_h3,points_h3,overview_page_views_h3,analysis_page_views_h3,markets_page_views_h3,charts_page_views_h3,trades_page_views_h3,forum_page_views_h3,influence_page_views_h3,total_page_views_h3,fb_likes_h3,fb_talking_about_h3,twitter_followers_h3,twitter_following_h3,twitter_lists_h3,twitter_favourites_h3,twitter_statuses_h3,reddit_subscribers_h3,reddit_active_users_h3,reddit_posts_per_hour_h3,reddit_posts_per_day_h3,reddit_comments_per_hour_h3,reddit_comments_per_day_h3,code_repo_stars_h3,code_repo_forks_h3,code_repo_subscribers_h3,code_repo_open_pull_issues_h3,code_repo_closed_pull_issues_h3,code_repo_open_issues_h3,code_repo_closed_issues_h3,code_repo_contributors_h3,date_h3,date_utc_h3,coin_name_h3,coin_id_h3,comments_h4,posts_h4,followers_h4,points_h4,overview_page_views_h4,analysis_page_views_h4,markets_page_views_h4,charts_page_views_h4,trades_page_views_h4,forum_page_views_h4,influence_page_views_h4,total_page_views_h4,fb_likes_h4,fb_talking_about_h4,twitter_f

Addiitional steps to process data:
* Group the data by types of social media

In [15]:
code = ['code_repo_stars',
'code_repo_forks' ,
'code_repo_subscribers',
'code_repo_contributors',
'code_repo_open_pull_issues',
'code_repo_closed_pull_issues',
'code_repo_open_issues',
'code_repo_closed_issues']

twitter = ['twitter_followers',
'twitter_following',
'twitter_lists',
'twitter_favourites',
'twitter_statuses']

reddit = ['reddit_subscribers',
'reddit_active_users',
'reddit_posts_per_hour',
'reddit_posts_per_day',
'reddit_comments_per_hour',
'reddit_comments_per_day'
]

fb = ['fb_likes','fb_talking_about']

market = [
'comments',
'posts',
'followers' ,
'points',
'overview_page_views',
'analysis_page_views',
'markets_page_views',
'charts_page_views',
'trades_page_views',
'forum_page_views',
'influence_page_views',
'total_page_views'
]


In [16]:
h = 49
col_dict = dict()
for f in ['code','twitter','reddit','fb','market']:
    dt = dict()
    for i in eval(f):
        dt[i] = [i+"_h"+str(y) for y in range(h)]
    col_dict[f] = dt

In [26]:
core = [
        #'channelLink','channelTitle','signalTime'*/
        'currency','exchange',
        'timestamp','pumptime_edited','coin_name', 'coin_id']

In [27]:
col_dict['core'] = core

In [28]:
col_dict[list(col_dict.keys())[0]].keys()

dict_keys(['code_repo_stars', 'code_repo_forks', 'code_repo_subscribers', 'code_repo_contributors', 'code_repo_open_pull_issues', 'code_repo_closed_pull_issues', 'code_repo_open_issues', 'code_repo_closed_issues'])

In [30]:
code_d = [x+"_h"+str(y) for x in code for y in range(h)]
twitter_d = [x+"_h"+str(y) for x in twitter for y in range(h)]
reddit_d = [x+"_h"+str(y) for x in reddit for y in range(h)]
fb_d = [x+"_h"+str(y) for x in fb for y in range(h)]
market_d = [x+"_h"+str(y) for x in market for y in range(h)]
timestamp = ['timestamp'+'_h'+str(y)  for y in range(h)]

In [31]:
#Omit timestamp for now
edited_cols = np.array(core + code_d + twitter_d + reddit_d + fb_d + market_d)
#MUST MAKE SURE THE ELEMENTS ARE IN THE SAME ORDER AS IN COL_DICT ABOVE!!!!
social_features = np.array(code_d + twitter_d + reddit_d + fb_d + market_d)
print(len(edited_cols))
unique_cols = np.unique(edited_cols)
print(len(unique_cols))

1623
1623


## Make sure all the importmnat columns are preserved

In [32]:
orig_cols = x_final.columns
for i in orig_cols:
    if i not in unique_cols:
        print(i)

event_date
event_time
timestamp_h0
timestamp_h1
timestamp_h2
timestamp_h3
timestamp_h4
timestamp_h5
timestamp_h6
timestamp_h7
timestamp_h8
timestamp_h9
timestamp_h10
timestamp_h11
timestamp_h12
timestamp_h13
timestamp_h14
timestamp_h15
timestamp_h16
timestamp_h17
timestamp_h18
timestamp_h19
timestamp_h20
timestamp_h21
timestamp_h22
timestamp_h23
timestamp_h24
timestamp_h25
timestamp_h26
timestamp_h27
timestamp_h28
timestamp_h29
timestamp_h30
timestamp_h31
timestamp_h32
timestamp_h33
timestamp_h34
timestamp_h35
timestamp_h36
timestamp_h37
timestamp_h38
timestamp_h39
timestamp_h40
timestamp_h41
timestamp_h42
timestamp_h43
timestamp_h44
timestamp_h45
timestamp_h46
timestamp_h47
timestamp_h48
date_h0
date_utc_h0
coin_name_h0
coin_id_h0
date_h1
date_utc_h1
coin_name_h1
coin_id_h1
date_h2
date_utc_h2
coin_name_h2
coin_id_h2
date_h3
date_utc_h3
coin_name_h3
coin_id_h3
date_h4
date_utc_h4
coin_name_h4
coin_id_h4
date_h5
date_utc_h5
coin_name_h5
coin_id_h5
date_h6
date_utc_h6
coin_name_h6
coin_

In [33]:
x_final = x_final[edited_cols]
x_final = x_final.reset_index(drop=True)

In [34]:
x_final.to_pickle("social_data_training_h.pkl")

In [35]:
x_final.head()

,currency,exchange,timestamp,pumptime_edited,coin_name,coin_id,code_repo_stars_h0,code_repo_stars_h1,code_repo_stars_h2,code_repo_stars_h3,code_repo_stars_h4,code_repo_stars_h5,code_repo_stars_h6,code_repo_stars_h7,code_repo_stars_h8,code_repo_stars_h9,code_repo_stars_h10,code_repo_stars_h11,code_repo_stars_h12,code_repo_stars_h13,code_repo_stars_h14,code_repo_stars_h15,code_repo_stars_h16,code_repo_stars_h17,code_repo_stars_h18,code_repo_stars_h19,code_repo_stars_h20,code_repo_stars_h21,code_repo_stars_h22,code_repo_stars_h23,code_repo_stars_h24,code_repo_stars_h25,code_repo_stars_h26,code_repo_stars_h27,code_repo_stars_h28,code_repo_stars_h29,code_repo_stars_h30,code_repo_stars_h31,code_repo_stars_h32,code_repo_stars_h33,code_repo_stars_h34,code_repo_stars_h35,code_repo_stars_h36,code_repo_stars_h37,code_repo_stars_h38,code_repo_stars_h39,code_repo_stars_h40,code_repo_stars_h41,code_repo_stars_h42,code_repo_stars_h43,code_repo_stars_h44,code_repo_stars_h45,code_repo_stars_h46,code_repo_stars_h47,code_repo_stars_h48,code_repo_forks_h0,code_repo_forks_h1,code_repo_forks_h2,code_repo_forks_h3,code_repo_forks_h4,code_repo_forks_h5,code_repo_forks_h6,code_repo_forks_h7,code_repo_forks_h8,code_repo_forks_h9,code_repo_forks_h10,code_repo_forks_h11,code_repo_forks_h12,code_repo_forks_h13,code_repo_forks_h14,code_repo_forks_h15,code_repo_forks_h16,code_repo_forks_h17,code_repo_forks_h18,code_repo_forks_h19,code_repo_forks_h20,code_repo_forks_h21,code_repo_forks_h22,code_repo_forks_h23,code_repo_forks_h24,code_repo_forks_h25,code_repo_forks_h26,code_repo_forks_h27,code_repo_forks_h28,code_repo_forks_h29,code_repo_forks_h30,code_repo_forks_h31,code_repo_forks_h32,code_repo_forks_h33,code_repo_forks_h34,code_repo_forks_h35,code_repo_forks_h36,code_repo_forks_h37,code_repo_forks_h38,code_repo_forks_h39,code_repo_forks_h40,code_repo_forks_h41,code_repo_forks_h42,code_repo_forks_h43,code_repo_forks_h44,code_repo_forks_h45,code_repo_forks_h46,code_repo_forks_h47,code_repo_forks_h48,code_repo_subscribers_h0,code_repo_subscribers_h1,code_repo_subscribers_h2,code_repo_subscribers_h3,code_repo_subscribers_h4,code_repo_subscribers_h5,code_repo_subscribers_h6,code_repo_subscribers_h7,code_repo_subscribers_h8,code_repo_subscribers_h9,code_repo_subscribers_h10,code_repo_subscribers_h11,code_repo_subscribers_h12,code_repo_subscribers_h13,code_repo_subscribers_h14,code_repo_subscribers_h15,code_repo_subscribers_h16,code_repo_subscribers_h17,code_repo_subscribers_h18,code_repo_subscribers_h19,code_repo_subscribers_h20,code_repo_subscribers_h21,code_repo_subscribers_h22,code_repo_subscribers_h23,code_repo_subscribers_h24,code_repo_subscribers_h25,code_repo_subscribers_h26,code_repo_subscribers_h27,code_repo_subscribers_h28,code_repo_subscribers_h29,code_repo_subscribers_h30,code_repo_subscribers_h31,code_repo_subscribers_h32,code_repo_subscribers_h33,code_repo_subscribers_h34,code_repo_subscribers_h35,code_repo_subscribers_h36,code_repo_subscribers_h37,code_repo_subscribers_h38,code_repo_subscribers_h39,code_repo_subscribers_h40,code_repo_subscribers_h41,code_repo_subscribers_h42,code_repo_subscribers_h43,code_repo_subscribers_h44,code_repo_subscribers_h45,code_repo_subscribers_h46,code_repo_subscribers_h47,code_repo_subscribers_h48,code_repo_contributors_h0,code_repo_contributors_h1,code_repo_contributors_h2,code_repo_contributors_h3,code_repo_contributors_h4,code_repo_contributors_h5,code_repo_contributors_h6,code_repo_contributors_h7,code_repo_contributors_h8,code_repo_contributors_h9,code_repo_contributors_h10,code_repo_contributors_h11,code_repo_contributors_h12,code_repo_contributors_h13,code_repo_contributors_h14,code_repo_contributors_h15,code_repo_contributors_h16,code_repo_contributors_h17,code_repo_contributors_h18,code_repo_contributors_h19,code_repo_contributors_h20,code_repo_contributors_h21,code_repo_contributors_h22,code_repo_contributors_h23,code_repo_contributors_h24,code_repo_contributors_h25,code_repo_contributors_h26,code_repo_cont

## Save the info

In [36]:
f = open("col_dict_h.pkl","wb")
pickle.dump(col_dict,f)
f.close()

g = open("social_features_h.pkl","wb")
pickle.dump(social_features,g)
g.close()